In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm, tqdm_notebook,tqdm_pandas
import itertools
import re

### sample 강릉

In [59]:
def count_word(df, tag, stopword, save_name, start_year=2010, end_year=2020, gap=1, month=False, months=[], save=True):
    '''단어가 등장한 횟수를 카운팅하고, 각 연도별 순위변동을 표시

    Parameters
    ----------
        df (dataFrame): DataFrame
        tag (string) : 품사지정
        stopword (list) : 카운팅에서 제외할 불용어 추가(유사검색어)
        save_name : 저장파일이름
        
        start_year (int) : 순위변동 확인시 시작할 연도위치
        end_year (int) : 순위변동 확인시 마지막 연도위치 
        gap (int): 연도별 기간차이 지정
        month (bool) : 특정월 추출여부
        months (list) : 추출할 월list
        save : 파일저장여부
       
    Returns
    -------
    vertical_df : DataFrame

    '''
    main_df = pd.DataFrame()

    if month == True:
        df = df[df.month.isin(months)]
    for s_year in range(2010, 2020):
        word_year_df = df[df['year'] == s_year]

        target_df = list(itertools.chain(*word_year_df[tag]))
        counted = pd.DataFrame(
            pd.Series(target_df).value_counts()).reset_index()
        counted.columns = ['name', 'value']

        drop_row = counted[counted['name'].isin(stopword)].index
        counted.drop(drop_row, inplace=True)
        counted = counted.head(15)

        counted['year'] = s_year
        counted['rank'] = counted.value.rank(ascending=False)
        main_df = pd.concat([main_df, counted])

    # gap
    main_df['gap'] = 0
    years = []
    for one_year in range(start_year, end_year, gap):
        years.append(one_year)
        names = main_df.loc[(main_df['year'] == one_year), "name"].values
        for name in names:
            try:
                new_rank = main_df.loc[(main_df['year'] == one_year) & (
                    main_df['name'] == name), 'rank'].values[0]
                old_rank = main_df.loc[(
                    main_df['year'] == one_year-gap) & (main_df['name'] == name), 'rank'].values[0]
                main_df.loc[(main_df['year'] == one_year) & (
                    main_df['name'] == name), 'gap'] = old_rank-new_rank
            except:
                main_df.loc[(main_df['year'] == one_year) & (
                    main_df['name'] == name), 'gap'] = "NEW"
    main_df = main_df[main_df['year'].isin(years)]
    main_df.drop('rank', axis=1, inplace=True)
    main_df.reset_index(drop = True, inplace = True)
    #vertical
    vertical_df = pd.DataFrame()
    for year in main_df.year.unique():
        sperated = main_df[main_df.year == year].reset_index(drop = True)
        vertical_df = pd.concat([vertical_df,sperated],axis = 1)

    if save == True:
        vertical_df.to_csv('./output/연도별카운트/{save_name}.csv'.format(
            save_name=save_name), encoding='cp949', index='False')

    return vertical_df

In [2]:
#for 돌리면 제외단어가 고정되는 불상사 발생, 딕셔너리 형태로 파일에 따른 스탑워드 지정이 필요할지도 
path = './output/token_통합/'
file_list = os.listdir(path)
all_df = pd.read_csv(path+file)
    

### 연도별 gap 확인

In [3]:
all_df = all_df.fillna("")
all_df['Noun'] = all_df['Noun'].apply(lambda x : re.sub("[\[\]' ]","",x).split(','))
all_df['Adjective'] = all_df['Adjective'].apply(lambda x : re.sub("[\[\]' ]","",x).split(','))

In [67]:
noun_df = count_word(all_df, 'Noun', ['강릉','강릉시','강원'],gap = 2, save_name = '2년 gap명사')
adj_df = count_word(all_df, 'Adjective', ['이다'],save_name = '1년 gap형용사')

In [65]:
#월[6,7,8] 여름기준
vertical_df = count_word(all_df, 'Noun', ['강릉','강릉시','강원','여름','거리'],gap = 1, save_name = '여름 1년gap명사',month = True, months = [6,7,8])

### 이하작업용

In [99]:
a = all_df[all_df.month.isin([6,7,8])]
a = a[a.year == 2010]
a['Noun']

805     [알바, 뭔가, 강릉, 청주, 강릉, 길이, 요새, 렌즈, 버릇, 중간, 중간, 카...
806     [강릉, 한참, 고민, 오야, 지인, 족발, 탕수육, 상호, 연락처, 옴니아, 인터...
807     [강릉단오제, 기간, 행사, 강원, 강릉시, 단오, 사당, 정보, 행사, 정보, 방...
808     [강릉, 경포대, 정동진, 초당, 순두부, 메뉴판, 동동주, 낚였다, 경포대, 압박...
809     [강릉오죽헌, 지정, 번호, 보물, 정일, 강릉시, 죽헌, 조선, 중기, 목조, 건...
                              ...                        
8192    [겨울, 개인, 사정, 어디, 변변찮, 우연, 여름, 휴가, 인천, 처형, 가족, ...
8193    [정동진, 버스, 강릉, 강릉, 버스, 터미널, 택시, 목장, 바람, 분다, 한숨,...
8194    [평창, 오대산, 펜션, 오대산, 전나무, 숲길, 월정사, 방아, 다리, 약수터, ...
8195    [지역별, 채용정보, 목요일, 모집, 내용, 회사, 나이, 마감, 이노, 검사, 조...
8196    [서울, 광화문, 세종, 문화, 회관, 지하철, 호선, 광화문역, 출구, 잠실, 롯...
Name: Noun, Length: 1442, dtype: object

In [100]:
target_df = list(itertools.chain(*a['Noun']))
counted = pd.DataFrame(
    pd.Series(target_df).value_counts()).reset_index()
counted.columns = ['name', 'value']

In [101]:
counted.head(10)

,name,value
0,강릉,2931
1,서울,1612
2,마을,1574
3,강원,1437
4,계곡,1345
5,한국,1025
6,해수욕장,948
7,동해,907
8,체험,858
9,문화,793


In [84]:
stopword = ['강릉','강릉시','강원']

In [90]:
drop_row = counted[counted['name'].isin(stopword)].index
counted.drop(drop_row, inplace=True)
counted = counted.head(15)

counted['year'] = s_year
counted['rank'] = counted.value.rank(ascending=False)
# main_df = pd.concat([main_df, counted])

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


NameError: name 'main_df' is not defined

In [91]:
counted

,name,value,year,rank
1,마을,1402,2010,1.0
2,커피,1249,2010,2.0
3,평창,1198,2010,3.0
4,서울,1056,2010,4.0
5,코스,1010,2010,5.0
7,이야기,998,2010,6.0
8,펜션,978,2010,7.0
9,한국,894,2010,8.0
10,해수욕장,880,2010,9.0
11,동해,863,2010,10.0
